In [233]:
data_path = "/content/drive/MyDrive/ML_Datasets/EEG-EyeBlinks-public/EEG-EyeBlinks"

IO_path = "/content/drive/MyDrive/ML_Datasets/EEG-EyeBlinks-public/EEG-EyeBlinks/EEG-IO"
# 0 to 19 trailel
VR_path = "/content/drive/MyDrive/ML_Datasets/EEG-EyeBlinks-public/EEG-EyeBlinks/EEG-VR"
# 0 to 12 trailel
VV_path = "/content/drive/MyDrive/ML_Datasets/EEG-EyeBlinks-public/EEG-EyeBlinks/EEG-VV"
# 0 to 12 trailel

Data_path = "/content/drive/MyDrive/ML_Datasets/EEG-EyeBlinks-public/EEG-EyeBlinks/EEG-IO/S00_data.csv"
Labels_path = "/content/drive/MyDrive/ML_Datasets/EEG-EyeBlinks-public/EEG-EyeBlinks/EEG-IO/S00_labels.csv"


In [234]:
# ==============================
# 📚 جميع المكتبات المطلوبة
# ==============================

# التعامل مع الملفات والبيانات
import os               # للتعامل مع الملفات والمجلدات
import csv              # قراءة ملفات CSV
import numpy as np      # المصفوفات وعمليات الحساب
import random           # توليد نوافذ عشوائية أو shuffle

# معالجة الإشارات (Signal Processing)
from scipy.signal import butter, lfilter   # تطبيق فلتر Low-pass / High-pass

# بناء الشبكات العصبية (Deep Learning)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# تقييم الأداء وVisualization
import matplotlib.pyplot as plt                      # رسم الإشارات والنتائج
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# تحسين وتحميل البيانات
from sklearn.utils import shuffle                    # لخلط البيانات قبل التدريب



In [235]:
def decode_stim(data_path, file_stim):
    interval_corrupt = []
    blinks = []
    n_corrupt = 0
    with open(os.path.join(data_path,file_stim)) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            if row[0]=="corrupt":
                n_corrupt = int(row[1])
            elif n_corrupt > 0:
                if float(row[1]) == -1:
                    t_end = data_sig[-1,0]
                else:
                    t_end = float(row[1])
                interval_corrupt.append([float(row[0]), t_end])
                n_corrupt = n_corrupt - 1
            elif row[0]=="blinks":
                #check that n_corrupt is 0
                if not n_corrupt==0:
                    print ("!Error in parsing")
            else:
                blinks.append([float(row[0]), int(row[1])])
    blinks = np.array(blinks)

    return interval_corrupt, blinks

In [236]:
import os
import numpy as np

fs = 250.0  # Sampling frequency

def load_full_dataset(folder_path):
    """
    Reads all *_data.csv and *_labels.csv files in the folder
    Supports EEG-IO, EEG-MB, EEG-VR, EEG-VV
    Returns:
        all_data: list of numpy arrays (each subject's data)
        all_blinks: list of lists (blink labels per subject)
        all_intervals: list of intervals per subject
        all_subject_names: list of subject names
    """

    data_files = sorted([f for f in os.listdir(folder_path) if '_data' in f])

    all_data = []
    all_blinks = []
    all_intervals = []
    all_subject_names = []

    for data_file in data_files:
        label_file = data_file.replace('_data', '_labels')
        data_path = os.path.join(folder_path, data_file)

        print(f"\nProcessing: {data_file}")

        # ----- Load data according to dataset type -----
        if folder_path.endswith('EEG-IO') or folder_path.endswith('EEG-MB'):
            data_sig = np.loadtxt(data_path, delimiter=";", skiprows=1, usecols=(0,1,2))
        elif folder_path.endswith('EEG-VR') or folder_path.endswith('EEG-VV'):
            data_sig = np.loadtxt(data_path, delimiter=",", skiprows=5, usecols=(0,1,2))
            data_sig = data_sig[0:(int(200*fs)+1), :]
            data_sig = data_sig[:, 0:3]
            data_sig[:,0] = np.arange(0, len(data_sig)) / fs
        else:
            raise ValueError("Unknown dataset folder. Supported: EEG-IO, EEG-MB, EEG-VR, EEG-VV")

        # ----- Load labels -----
        intervals, blinks = decode_stim(folder_path, label_file)

        # ----- Store -----
        all_data.append(data_sig)
        all_blinks.append(blinks)
        all_intervals.append(intervals)
        all_subject_names.append(data_file.replace("_data.csv",""))

        print(f"  - Data shape: {data_sig.shape}")
        print(f"  - Blinks: {len(blinks)}")
        print(f"  - Corrupt intervals: {len(intervals)}")

    print("\nFinished loading all dataset!")
    return all_data, all_blinks, all_intervals, all_subject_names


In [237]:
IO_all_data, IO_all_blinks, IO_all_intervals, IO_all_subject_names = load_full_dataset(IO_path)


Processing: S00_data.csv
  - Data shape: (25568, 3)
  - Blinks: 25
  - Corrupt intervals: 1

Processing: S01_data.csv
  - Data shape: (25568, 3)
  - Blinks: 26
  - Corrupt intervals: 0

Processing: S02_data.csv
  - Data shape: (25568, 3)
  - Blinks: 25
  - Corrupt intervals: 0

Processing: S03_data.csv
  - Data shape: (25568, 3)
  - Blinks: 26
  - Corrupt intervals: 1

Processing: S04_data.csv
  - Data shape: (25536, 3)
  - Blinks: 26
  - Corrupt intervals: 0

Processing: S05_data.csv
  - Data shape: (25568, 3)
  - Blinks: 26
  - Corrupt intervals: 0

Processing: S06_data.csv
  - Data shape: (19328, 3)
  - Blinks: 27
  - Corrupt intervals: 0

Processing: S07_data.csv
  - Data shape: (25696, 3)
  - Blinks: 26
  - Corrupt intervals: 0

Processing: S08_data.csv
  - Data shape: (25664, 3)
  - Blinks: 26
  - Corrupt intervals: 0

Processing: S09_data.csv
  - Data shape: (25696, 3)
  - Blinks: 26
  - Corrupt intervals: 0

Processing: S10_data.csv
  - Data shape: (25664, 3)
  - Blinks: 25
  

In [238]:
VR_all_data, VR_all_blinks, VR_all_intervals, VR_all_subject_names = load_full_dataset(VR_path)


Processing: S00R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 61
  - Corrupt intervals: 0

Processing: S01R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 29
  - Corrupt intervals: 1

Processing: S02R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 26
  - Corrupt intervals: 0

Processing: S03R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 96
  - Corrupt intervals: 1

Processing: S04R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 101
  - Corrupt intervals: 6

Processing: S05R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 27
  - Corrupt intervals: 3

Processing: S06R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 16
  - Corrupt intervals: 0

Processing: S07R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 111
  - Corrupt intervals: 2

Processing: S08R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 49
  - Corrupt intervals: 2

Processing: S09R_data.csv
  - Data shape: (50001, 3)
  - Blinks: 11
  - Corrupt intervals: 0

Processing: S10R_data.csv
  - Data shape: (50001, 3)
  - 

In [239]:
VV_all_data, VV_all_blinks, VV_all_intervals, VV_all_subject_names = load_full_dataset(VV_path)


Processing: S00V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 66
  - Corrupt intervals: 0

Processing: S01V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 16
  - Corrupt intervals: 0

Processing: S02V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 36
  - Corrupt intervals: 1

Processing: S03V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 74
  - Corrupt intervals: 2

Processing: S04V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 102
  - Corrupt intervals: 2

Processing: S05V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 42
  - Corrupt intervals: 1

Processing: S06V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 16
  - Corrupt intervals: 0

Processing: S07V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 132
  - Corrupt intervals: 1

Processing: S08V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 71
  - Corrupt intervals: 1

Processing: S09V_data.csv
  - Data shape: (50001, 3)
  - Blinks: 43
  - Corrupt intervals: 1

Processing: S10V_data.csv
  - Data shape: (50001, 3)
  - 

In [240]:
# Positive: Startle Blinks (code=1)
positive_blinks = []

for i, blinks in enumerate(IO_all_blinks):
    startle_idx = np.where(blinks[:,1] == 1)[0]
    positive_blinks.extend([ (i, idx) for idx in startle_idx ])


In [241]:
import random

# Negative:
# 50% نوافذ عشوائية من IO (بعيدة عن blink)
negative_windows = []
window_size = 1

for i, data in enumerate(IO_all_data):
    # نختار n نافذة بعشوائية من كل subject
    n_windows = len(IO_all_blinks[i])  # تقريبًا نفس عدد البوزيتيف
    max_idx = data.shape[0] - window_size
    random_idx = random.sample(range(max_idx), n_windows)
    negative_windows.extend([ (i, idx) for idx in random_idx ])

# 30% و 20% من blinks كود 0 و 2 من VR + VV
vr_neg = []
for i, blinks in enumerate(VR_all_blinks):
    neg_idx = np.where((blinks[:,1]==0) | (blinks[:,1]==2))[0]
    vr_neg.extend([ (i, idx) for idx in neg_idx ])

vv_neg = []
for i, blinks in enumerate(VV_all_blinks):
    neg_idx = np.where((blinks[:,1]==0) | (blinks[:,1]==2))[0]
    vv_neg.extend([ (i, idx) for idx in neg_idx ])


In [242]:
import numpy as np

def create_dataset_from_indices(all_data, all_blinks, positive_indices, negative_indices, window_size=250):
    """
    all_data: list of arrays per subject (samples x channels)
    all_blinks: list of arrays per subject (blink_time, code)
    positive_indices: list of tuples (subject_idx, blink_idx)
    negative_indices: list of tuples (subject_idx, start_idx)
    """
    X = []
    y = []

    # Positive windows
    for subj_idx, blink_idx in positive_indices:
        data = all_data[subj_idx]
        blink_time = all_blinks[subj_idx][blink_idx, 0]  # time in sec
        start_sample = int(blink_time * fs) - window_size//2
        start_sample = max(0, start_sample)
        end_sample = start_sample + window_size
        if end_sample <= data.shape[0]:
            X.append(data[start_sample:end_sample, 1:])  # channels only
            y.append(1)

    # Negative windows
    for subj_idx, start_idx in negative_indices:
        data = all_data[subj_idx]
        end_idx = start_idx + window_size
        if end_idx <= data.shape[0]:
            X.append(data[start_idx:end_idx, 1:])
            y.append(0)

    return np.array(X), np.array(y)

# مثال للاستخدام
X_posneg, y_posneg = create_dataset_from_indices(
    all_data=IO_all_data + VR_all_data + VV_all_data,
    all_blinks=IO_all_blinks + VR_all_blinks + VV_all_blinks,
    positive_indices=positive_blinks,
    negative_indices=negative_windows + vr_neg + vv_neg,
    window_size=250  # 1 sec
)

print("Shape of X:", X_posneg.shape)
print("Shape of y:", y_posneg.shape)


Shape of X: (2507, 250, 2)
Shape of y: (2507,)


In [243]:
from sklearn.utils import shuffle

X_shuffled, y_shuffled = shuffle(X_posneg, y_posneg, random_state=42)


In [244]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X_shuffled, y_shuffled, test_size=0.3, random_state=42, stratify=y_shuffled)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("Train shape:", X_train.shape)
print("Val shape:", X_val.shape)
print("Test shape:", X_test.shape)


Train shape: (1754, 250, 2)
Val shape: (376, 250, 2)
Test shape: (377, 250, 2)


In [245]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')


In [246]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


In [247]:
window_size = X_train.shape[1]  # 250 samples (1 sec)
n_channels = X_train.shape[2]   # 2 channels: Fp1, Fp2

inputs = Input(shape=(window_size, n_channels))
x = Conv1D(32, kernel_size=5, activation='relu', padding='same')(inputs)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(64, kernel_size=5, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
x = MaxPooling1D(pool_size=2)(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 250, 2)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_20 (Conv1D)              │ (None, 250, 32)        │           352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_19 (MaxPooling1D) │ (None, 125, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_21 (Conv1D)              │ (None, 125, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_20 (MaxPooling1D) │ (None, 62, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_22 (Conv1D)              │ (None, 62, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_21 (MaxPooling1D) │ (None, 31, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 3968)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │       254,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 289,441 (1.10 MB)

 Trainable params: 289,441 (1.10 MB)

 Non-trainable params: 0 (0.00 B)

In [248]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_cnn_model.h5', monitor='val_loss', save_best_only=True)
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=callbacks
)


Epoch 1/30
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6932 - loss: 411.4105

55/55 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.6946 - loss: 402.1575 - val_accuracy: 0.8112 - val_loss: 0.6984
Epoch 2/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8114 - loss: 0.7471

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8114 - loss: 0.7465 - val_accuracy: 0.8112 - val_loss: 0.6685
Epoch 3/30
53/55 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8100 - loss: 0.6645

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8101 - loss: 0.6643 - val_accuracy: 0.8112 - val_loss: 0.6521
Epoch 4/30
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7994 - loss: 0.6503

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7999 - loss: 0.6501 - val_accuracy: 0.8112 - val_loss: 0.6376
Epoch 5/30
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8146 - loss: 0.6337

...

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8075 - loss: 0.5061 - val_accuracy: 0.8112 - val_loss: 0.5009
Epoch 27/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8182 - loss: 0.4933

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8181 - loss: 0.4934 - val_accuracy: 0.8112 - val_loss: 0.4991
Epoch 28/30
54/55 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7989 - loss: 0.5115

55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.7994 - loss: 0.5110 - val_accuracy: 0.8112 - val_loss: 0.4975
Epoch 29/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.8042 - loss: 0.5046

55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.8043 - loss: 0.5044 - val_accuracy: 0.8112 - val_loss: 0.4960
Epoch 30/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8054 - loss: 0.5020

55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.8055 - loss: 0.5018 - val_accuracy: 0.8112 - val_loss: 0.4947


In [249]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import numpy as np


In [250]:
# الحصول على probabilities
y_pred_prob = model.predict(X_test)

# تحويل probabilities لـ 0 أو 1
threshold = 0.2
y_pred = (y_pred_prob >= threshold).astype(int)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


In [251]:
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision*100:.2f}%")
print(f"Recall: {recall*100:.2f}%")
print(f"F1-score: {f1*100:.2f}%")


Precision: 18.83%
Recall: 100.00%
F1-score: 31.70%


In [252]:
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)


Confusion Matrix:
[[  0 306]
 [  0  71]]


In [253]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

# y_test : الـ ground truth (0 أو 1)
# y_pred_prob : probabilities الناتجة من الموديل

thresholds = np.arange(0.1, 1.0, 0.05)
best_f1 = 0
best_thresh = 0.5
results = []

for t in thresholds:
    y_pred = (y_pred_prob >= t).astype(int)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    results.append((t, precision, recall, f1))

    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"Best threshold: {best_thresh}")
print(f"Precision: {results[int((best_thresh-0.1)/0.05)][1]*100:.2f}%")
print(f"Recall: {results[int((best_thresh-0.1)/0.05)][2]*100:.2f}%")
print(f"F1-score: {results[int((best_thresh-0.1)/0.05)][3]*100:.2f}%")

# لو حابب، اطبع كل النتائج
for r in results:
    print(f"Threshold: {r[0]:.2f}, Precision: {r[1]*100:.2f}%, Recall: {r[2]*100:.2f}%, F1: {r[3]*100:.2f}%")


Best threshold: 0.1
Precision: 18.83%
Recall: 100.00%
F1-score: 31.70%
Threshold: 0.10, Precision: 18.83%, Recall: 100.00%, F1: 31.70%
Threshold: 0.15, Precision: 18.83%, Recall: 100.00%, F1: 31.70%
Threshold: 0.20, Precision: 18.83%, Recall: 100.00%, F1: 31.70%
Threshold: 0.25, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.30, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.35, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.40, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.45, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.50, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.55, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.60, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.65, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.70, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.75, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.80, Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Threshold: 0.85, 

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.p